![DataStax Academy](https://s3.amazonaws.com/datastaxtraining/vq8Jr36Gk48v/datastax-academy.svg "DataStax Academy")

# Exercise 08.07 - Spark SQL: Dataframe Language Integrated

## Background

In this example we will execute some additional examples of SQL queries.

***

## Directions

A useful tool is the `registerTempTable` method. We will begin by creating a dataframe.

In [5]:
val maxDF = sqlContext.sql(
    "SELECT actor_name, MAX(release_year) as max_year " + 
    "FROM killr_video.videos_by_actor " +
    "WHERE actor_name = 'Johnny Depp' " + 
    "GROUP BY actor_name")

maxDF.show

+-----------+--------+
| actor_name|max_year|
+-----------+--------+
|Johnny Depp|    2015|
+-----------+--------+



Now we will register that information as a temp table and use it in a SQL join.

In [6]:
maxDF.registerTempTable("max_year")

In [10]:
val movieDF = sqlContext.sql(" SELECT M.actor_name, V.title, V.release_year" + 
                             " FROM max_year as M JOIN killr_video.videos_by_actor as V" +
                             " ON M.actor_name = V.actor_name" + 
                             " WHERE V.release_year > M.max_year - 1" + 
                             " ORDER BY release_year DESC")
movieDF.show

+-----------+----------+------------+
| actor_name|     title|release_year|
+-----------+----------+------------+
|Johnny Depp|Black Mass|        2015|
+-----------+----------+------------+



Another interesting method you can use is the `explain` method. This will output the query execution plan, so you can follow the steps Spark is using to complete the task.

In [11]:
movieDF.explain

== Physical Plan ==
Sort [release_year#34 DESC], true
 Exchange (RangePartitioning 200)
  Project [actor_name#7,title#36,release_year#34]
   Filter (release_year#34 > (max_year#6 - 1))
    ShuffledHashJoin [actor_name#7], [actor_name#33], BuildRight
     Aggregate false, [actor_name#7], [actor_name#7,MAX(PartialMax#39) AS max_year#6]
      Exchange (HashPartitioning 200)
       Aggregate true, [actor_name#7], [actor_name#7,MAX(release_year#8) AS PartialMax#39]
        Filter (actor_name#7 = Johnny Depp)
         HiveTableScan [actor_name#7,release_year#8], (MetastoreRelation killr_video, videos_by_actor, None), None
     Exchange (HashPartitioning 200)
      HiveTableScan [actor_name#33,title#36,release_year#34], (MetastoreRelation killr_video, videos_by_actor, Some(V)), None
